# Bird Migration Predictive Modeling

## Comprehensive ML Models for Migration Prediction

This notebook builds multiple machine learning models to predict bird migration metrics:
- **Regression targets**: peak_speed_mph, peak_altitude_ft, total_passed, peak_birds
- **Classification target**: peak_direction
- **Model types**: Spline Regression, Random Forest, XGBoost
- **Analysis scope**: State-specific models (NJ, VT) and combined data models

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style("whitegrid")


In [30]:
# Load Data
data_path = Path("../data/processed_data/birdcast+viirs+weather/merged_dataset_with_weather.csv")
df = pd.read_csv(data_path)

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

print(f"Dataset shape: {df.shape}")
print(f"States: {df['state'].unique()}")
print(f"\n✓ Data loaded successfully")

# ====================================================
# DATA PREPARATION
# ====================================================

# Define features (exclude target variables and identifiers)
exclude_cols = ['date', 'state', 'peak_birds', 'total_passed', 'peak_speed_mph', 
                'peak_altitude_ft', 'peak_direction', 'year_month', 'season']

# Get numeric and categorical columns separately
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
feature_cols = [col for col in numeric_cols if col not in exclude_cols]

print(f"\nNumeric feature columns: {len(feature_cols)}")
print(feature_cols)

Dataset shape: (1542, 33)
States: ['NJ' 'VT']

✓ Data loaded successfully

Numeric feature columns: 25
['gapfilled_ntl', 'ntl_variability', 'lunar_irradiance', 'quality_score', 'day_of_year', 'month', 'year', 'peak_speed_missing', 'peak_altitude_missing', 'temperature_2m (°C)', 'relative_humidity_2m (%)', 'apparent_temperature (°C)', 'cloud_cover (%)', 'surface_pressure (hPa)', 'wind_speed_10m (km/h)', 'wind_direction_10m (°)', 'dew_point_2m (°C)', 'soil_temperature_0_to_7cm (°C)', 'rain (mm)', 'snowfall (cm)', 'wind_gusts_10m (km/h)', 'wind_speed_100m (km/h)', 'wind_direction_100m (°)', 'vapour_pressure_deficit (kPa)', 'hour']


In [31]:
# Data cleaning: Handle missing values and prepare data
# STRATEGY: Replace -1 sentinels with NaN, then impute with median + use missing flags
# This is mathematically correct: -1 is not a real physical value, NaN is the proper representation

df_model = df.dropna(subset=feature_cols + ['peak_birds', 'total_passed'])

# Replace -1 sentinels with NaN for speed and altitude
df_model['peak_speed_mph'] = df_model['peak_speed_mph'].replace(-1, np.nan)
df_model['peak_altitude_ft'] = df_model['peak_altitude_ft'].replace(-1, np.nan)

# The missing flags are already in the data (peak_speed_missing, peak_altitude_missing)
# These flags preserve the information that data was missing
print(f"\nTarget variable distributions (after replacing -1 with NaN):")
print(f"peak_birds - min: {df_model['peak_birds'].min()}, max: {df_model['peak_birds'].max()}")
print(f"total_passed - min: {df_model['total_passed'].min()}, max: {df_model['total_passed'].max()}")
print(f"peak_speed_mph - min: {df_model['peak_speed_mph'].min():.2f}, max: {df_model['peak_speed_mph'].max():.2f} (NaN instead of -1)")
print(f"peak_altitude_ft - min: {df_model['peak_altitude_ft'].min():.2f}, max: {df_model['peak_altitude_ft'].max():.2f} (NaN instead of -1)")
print(f"peak_direction classes: {df_model['peak_direction'].nunique()}")
print(df_model['peak_direction'].value_counts().head(10))

print(f"\nMissing data indicators (preserved as features):")
print(f"peak_speed_missing flag: {df_model['peak_speed_missing'].sum()} records with missing speed")
print(f"peak_altitude_missing flag: {df_model['peak_altitude_missing'].sum()} records with missing altitude")

# For regression targets with NaN: impute with median
df_model_reg = df_model.copy()
df_model_reg['peak_speed_mph'] = df_model_reg['peak_speed_mph'].fillna(df_model_reg['peak_speed_mph'].median())
df_model_reg['peak_altitude_ft'] = df_model_reg['peak_altitude_ft'].fillna(df_model_reg['peak_altitude_ft'].median())

print(f"\nFinal dataset shape: {df_model_reg.shape}")
print(f"Missing flags as features: peak_speed_missing, peak_altitude_missing")
print(f"Imputation method: Median fill (avoids physically impossible -1 values)")
print(f"Speed median: {df_model['peak_speed_mph'].median():.2f} mph")
print(f"Altitude median: {df_model['peak_altitude_ft'].median():.2f} ft")

print(f"\n✓ Data preparation complete (NaN + imputation + missing flags)")


Target variable distributions (after replacing -1 with NaN):
peak_birds - min: 300.0, max: 10023300.0
total_passed - min: 0.0, max: 25221600.0
peak_speed_mph - min: 0.00, max: 59.00 (NaN instead of -1)
peak_altitude_ft - min: 500.00, max: 5900.00 (NaN instead of -1)
peak_direction classes: 17
peak_direction
Low activity    330
NE              212
NNE             179
SSW             169
SW              168
S               125
SSE              69
ENE              62
SE               60
WSW              59
Name: count, dtype: int64

Missing data indicators (preserved as features):
peak_speed_missing flag: 330 records with missing speed
peak_altitude_missing flag: 330 records with missing altitude

Final dataset shape: (1542, 33)
Missing flags as features: peak_speed_missing, peak_altitude_missing
Imputation method: Median fill (avoids physically impossible -1 values)
Speed median: 21.00 mph
Altitude median: 1700.00 ft

✓ Data preparation complete (NaN + imputation + missing flags)


## 1. Helper Functions for Model Training

### Functions for regression and classification model evaluation

In [32]:
def train_regression_models(X_train, X_test, y_train, y_test, target_name, state_label=""):
    """
    Train and evaluate multiple regression models: Spline, Random Forest, and XGBoost
    """
    results = {}
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # 1. Spline Regression (Polynomial Regression with degree 3)
    print(f"\n{'='*70}")
    print(f"Spline Regression - {target_name} {state_label}")
    print(f"{'='*70}")
    
    # Use polynomial features with degree 3 for spline-like behavior
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.pipeline import Pipeline
    
    spline_model = Pipeline([
        ('poly_features', PolynomialFeatures(degree=3, include_bias=False)),
        ('scaler', StandardScaler()),
        ('linear_regression', Ridge(alpha=1.0))  # Use Ridge to prevent overfitting
    ])
    
    spline_model.fit(X_train, y_train)
    spline_pred = spline_model.predict(X_test)
    spline_mse = mean_squared_error(y_test, spline_pred)
    spline_r2 = r2_score(y_test, spline_pred)
    spline_rmse = np.sqrt(spline_mse)
    
    print(f"RMSE: {spline_rmse:.4f}")
    print(f"R² Score: {spline_r2:.4f}")
    
    results['Spline'] = {'model': spline_model, 'pred': spline_pred, 'rmse': spline_rmse, 'r2': spline_r2}
    
    # 2. Random Forest
    print(f"\nRandom Forest - {target_name} {state_label}")
    print(f"{'='*70}")
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    rf_pred = rf_model.predict(X_test)
    rf_mse = mean_squared_error(y_test, rf_pred)
    rf_r2 = r2_score(y_test, rf_pred)
    rf_rmse = np.sqrt(rf_mse)
    
    print(f"RMSE: {rf_rmse:.4f}")
    print(f"R² Score: {rf_r2:.4f}")
    
    results['Random Forest'] = {'model': rf_model, 'pred': rf_pred, 'rmse': rf_rmse, 'r2': rf_r2}
    
    # 3. XGBoost
    print(f"\nXGBoost - {target_name} {state_label}")
    print(f"{'='*70}")
    xgb_model = xgb.XGBRegressor(n_estimators=100, max_depth=6, learning_rate=0.1, 
                                  random_state=42, verbosity=0)
    xgb_model.fit(X_train, y_train, verbose=False)
    xgb_pred = xgb_model.predict(X_test)
    xgb_mse = mean_squared_error(y_test, xgb_pred)
    xgb_r2 = r2_score(y_test, xgb_pred)
    xgb_rmse = np.sqrt(xgb_mse)
    
    print(f"RMSE: {xgb_rmse:.4f}")
    print(f"R² Score: {xgb_r2:.4f}")
    
    results['XGBoost'] = {'model': xgb_model, 'pred': xgb_pred, 'rmse': xgb_rmse, 'r2': xgb_r2}
    
    # 4. Feature importance comparison (for tree-based models)
    print(f"\nTop 10 Important Features (Random Forest):")
    importance_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': rf_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    print(importance_df.head(10))
    
    results['feature_importance'] = importance_df
    
    return results, scaler, X_test_scaled

def train_classification_models(X_train, X_test, y_train, y_test, target_name, state_label=""):
    """
    Train and evaluate classification models for peak_direction
    """
    results = {}
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # 1. Random Forest Classifier
    print(f"\n{'='*70}")
    print(f"Random Forest Classifier - {target_name} {state_label}")
    print(f"{'='*70}")
    rf_clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf_clf.fit(X_train, y_train)
    rf_pred = rf_clf.predict(X_test)
    rf_acc = accuracy_score(y_test, rf_pred)
    
    print(f"Accuracy: {rf_acc:.4f}")
    print(f"\nClassification Report:\n{classification_report(y_test, rf_pred, zero_division=0)}")
    
    results['Random Forest'] = {'model': rf_clf, 'pred': rf_pred, 'accuracy': rf_acc}
    
    # 2. XGBoost Classifier
    print(f"\nXGBoost Classifier - {target_name} {state_label}")
    print(f"{'='*70}")
    xgb_clf = xgb.XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1, 
                                 random_state=42, verbosity=0)
    xgb_clf.fit(X_train, y_train, verbose=False)
    xgb_pred = xgb_clf.predict(X_test)
    xgb_acc = accuracy_score(y_test, xgb_pred)
    
    print(f"Accuracy: {xgb_acc:.4f}")
    print(f"\nClassification Report:\n{classification_report(y_test, xgb_pred, zero_division=0)}")
    
    results['XGBoost'] = {'model': xgb_clf, 'pred': xgb_pred, 'accuracy': xgb_acc}
    
    # Feature importance
    print(f"\nTop 10 Important Features (Random Forest):")
    importance_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': rf_clf.feature_importances_
    }).sort_values('Importance', ascending=False)
    print(importance_df.head(10))
    
    results['feature_importance'] = importance_df
    
    return results, scaler, X_test_scaled

print("✓ Helper functions defined")

✓ Helper functions defined


## 2. Regression Models: peak_birds

### Combined Data (Both NJ and VT)

In [33]:
# Prepare data for peak_birds regression
X = df_model[feature_cols].copy()
y = df_model['peak_birds'].copy()

# Train-test split (combined data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape[0]}, Test set: {X_test.shape[0]}")

# Train models
results_pb_combined, scaler_pb, X_test_scaled_pb = train_regression_models(
    X_train, X_test, y_train, y_test, 
    target_name="peak_birds",
    state_label="(Combined)"
)

Training set: 1233, Test set: 309

Spline Regression - peak_birds (Combined)
RMSE: 1600892.5141RMSE: 1600892.5141
R² Score: -0.7581

Random Forest - peak_birds (Combined)

R² Score: -0.7581

Random Forest - peak_birds (Combined)
RMSE: 952633.2042
R² Score: 0.3775

XGBoost - peak_birds (Combined)
RMSE: 926752.7789
R² Score: 0.4108

Top 10 Important Features (Random Forest):
                          Feature  Importance
4                     day_of_year    0.164122
15         wind_direction_10m (°)    0.098520
22        wind_direction_100m (°)    0.085964
12                cloud_cover (%)    0.063541
21         wind_speed_100m (km/h)    0.059001
23  vapour_pressure_deficit (kPa)    0.055881
10       relative_humidity_2m (%)    0.049272
20          wind_gusts_10m (km/h)    0.037335
3                   quality_score    0.035821
9             temperature_2m (°C)    0.035718
RMSE: 952633.2042
R² Score: 0.3775

XGBoost - peak_birds (Combined)
RMSE: 926752.7789
R² Score: 0.4108

Top 10 Importa

### State-Specific Models

In [34]:
# NJ Model
print("\n" + "="*70)
print("NJ STATE MODEL")
print("="*70)
df_nj = df_model[df_model['state'] == 'NJ']
X_nj = df_nj[feature_cols].copy()
y_nj = df_nj['peak_birds'].copy()

X_train_nj, X_test_nj, y_train_nj, y_test_nj = train_test_split(X_nj, y_nj, test_size=0.2, random_state=42)
results_pb_nj, scaler_pb_nj, X_test_scaled_pb_nj = train_regression_models(
    X_train_nj, X_test_nj, y_train_nj, y_test_nj, 
    target_name="peak_birds",
    state_label="(NJ)"
)

# VT Model
print("\n" + "="*70)
print("VT STATE MODEL")
print("="*70)
df_vt = df_model[df_model['state'] == 'VT']
X_vt = df_vt[feature_cols].copy()
y_vt = df_vt['peak_birds'].copy()

X_train_vt, X_test_vt, y_train_vt, y_test_vt = train_test_split(X_vt, y_vt, test_size=0.2, random_state=42)
results_pb_vt, scaler_pb_vt, X_test_scaled_pb_vt = train_regression_models(
    X_train_vt, X_test_vt, y_train_vt, y_test_vt, 
    target_name="peak_birds",
    state_label="(VT)"
)


NJ STATE MODEL

Spline Regression - peak_birds (NJ)
RMSE: 11204795.7819
R² Score: -69.4825

Random Forest - peak_birds (NJ)
RMSE: 1117717.9800
R² Score: 0.2986

XGBoost - peak_birds (NJ)
RMSE: 1149845.6533
R² Score: 0.2577

Top 10 Important Features (Random Forest):
RMSE: 1117717.9800
R² Score: 0.2986

XGBoost - peak_birds (NJ)
RMSE: 1149845.6533
R² Score: 0.2577

Top 10 Important Features (Random Forest):
                          Feature  Importance
4                     day_of_year    0.166203
15         wind_direction_10m (°)    0.125884
22        wind_direction_100m (°)    0.093689
23  vapour_pressure_deficit (kPa)    0.072130
12                cloud_cover (%)    0.064384
10       relative_humidity_2m (%)    0.063390
13         surface_pressure (hPa)    0.057616
21         wind_speed_100m (km/h)    0.045328
20          wind_gusts_10m (km/h)    0.038511
3                   quality_score    0.031905

VT STATE MODEL

Spline Regression - peak_birds (VT)
RMSE: 3012369.1256
R² Score: -

## 3. Regression Models: total_passed

### Combined Data

In [35]:
# Combined
X = df_model[feature_cols].copy()
y = df_model['total_passed'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

results_tp_combined, scaler_tp, _ = train_regression_models(
    X_train, X_test, y_train, y_test, 
    target_name="total_passed",
    state_label="(Combined)"
)

# NJ
print("\n" + "="*70)
print("NJ STATE MODEL")
print("="*70)
X_nj = df_nj[feature_cols].copy()
y_nj = df_nj['total_passed'].copy()
X_train_nj, X_test_nj, y_train_nj, y_test_nj = train_test_split(X_nj, y_nj, test_size=0.2, random_state=42)
results_tp_nj, _, _ = train_regression_models(
    X_train_nj, X_test_nj, y_train_nj, y_test_nj, 
    target_name="total_passed",
    state_label="(NJ)"
)

# VT
print("\n" + "="*70)
print("VT STATE MODEL")
print("="*70)
X_vt = df_vt[feature_cols].copy()
y_vt = df_vt['total_passed'].copy()
X_train_vt, X_test_vt, y_train_vt, y_test_vt = train_test_split(X_vt, y_vt, test_size=0.2, random_state=42)
results_tp_vt, _, _ = train_regression_models(
    X_train_vt, X_test_vt, y_train_vt, y_test_vt, 
    target_name="total_passed",
    state_label="(VT)"
)


Spline Regression - total_passed (Combined)
RMSE: 2798145.3404
R² Score: -0.1157

Random Forest - total_passed (Combined)
RMSE: 2123993.9282
R² Score: 0.3572

XGBoost - total_passed (Combined)
RMSE: 2127783.6322
R² Score: 0.3549

Top 10 Important Features (Random Forest):
                          Feature  Importance
15         wind_direction_10m (°)    0.133001
4                     day_of_year    0.132989
22        wind_direction_100m (°)    0.132053
12                cloud_cover (%)    0.087384
21         wind_speed_100m (km/h)    0.053683
23  vapour_pressure_deficit (kPa)    0.049395
3                   quality_score    0.042658
13         surface_pressure (hPa)    0.040398
10       relative_humidity_2m (%)    0.040091
2                lunar_irradiance    0.038373

NJ STATE MODEL

Spline Regression - total_passed (NJ)
RMSE: 2123993.9282
R² Score: 0.3572

XGBoost - total_passed (Combined)
RMSE: 2127783.6322
R² Score: 0.3549

Top 10 Important Features (Random Forest):
              

## 4. Regression Models: peak_speed_mph

### Combined Data

In [36]:
# Combined (only valid speed values)
X = df_model_reg[feature_cols].copy()
y = df_model_reg['peak_speed_mph'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

results_ps_combined, scaler_ps, _ = train_regression_models(
    X_train, X_test, y_train, y_test, 
    target_name="peak_speed_mph",
    state_label="(Combined)"
)

# NJ
print("\n" + "="*70)
print("NJ STATE MODEL")
print("="*70)
df_nj_reg = df_model_reg[df_model_reg['state'] == 'NJ']
X_nj = df_nj_reg[feature_cols].copy()
y_nj = df_nj_reg['peak_speed_mph'].copy()
X_train_nj, X_test_nj, y_train_nj, y_test_nj = train_test_split(X_nj, y_nj, test_size=0.2, random_state=42)
results_ps_nj, _, _ = train_regression_models(
    X_train_nj, X_test_nj, y_train_nj, y_test_nj, 
    target_name="peak_speed_mph",
    state_label="(NJ)"
)

# VT
print("\n" + "="*70)
print("VT STATE MODEL")
print("="*70)
df_vt_reg = df_model_reg[df_model_reg['state'] == 'VT']
X_vt = df_vt_reg[feature_cols].copy()
y_vt = df_vt_reg['peak_speed_mph'].copy()
X_train_vt, X_test_vt, y_train_vt, y_test_vt = train_test_split(X_vt, y_vt, test_size=0.2, random_state=42)
results_ps_vt, _, _ = train_regression_models(
    X_train_vt, X_test_vt, y_train_vt, y_test_vt, 
    target_name="peak_speed_mph",
    state_label="(VT)"
)


Spline Regression - peak_speed_mph (Combined)
RMSE: 13.7556
R² Score: -1.3806

Random Forest - peak_speed_mph (Combined)
RMSE: 13.7556
R² Score: -1.3806

Random Forest - peak_speed_mph (Combined)
RMSE: 6.9870
R² Score: 0.3858

XGBoost - peak_speed_mph (Combined)
RMSE: 7.1393
R² Score: 0.3587

Top 10 Important Features (Random Forest):
RMSE: 6.9870
R² Score: 0.3858

XGBoost - peak_speed_mph (Combined)
RMSE: 7.1393
R² Score: 0.3587

Top 10 Important Features (Random Forest):
                    Feature  Importance
21   wind_speed_100m (km/h)    0.099656
4               day_of_year    0.077387
13   surface_pressure (hPa)    0.064777
15   wind_direction_10m (°)    0.064006
22  wind_direction_100m (°)    0.063063
2          lunar_irradiance    0.054933
16        dew_point_2m (°C)    0.053793
14    wind_speed_10m (km/h)    0.049446
3             quality_score    0.047313
12          cloud_cover (%)    0.046995

NJ STATE MODEL

Spline Regression - peak_speed_mph (NJ)
RMSE: 83.2439
R² Score: 

## 5. Regression Models: peak_altitude_ft

### Combined Data

In [37]:
# Combined
X = df_model_reg[feature_cols].copy()
y = df_model_reg['peak_altitude_ft'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

results_pa_combined, scaler_pa, _ = train_regression_models(
    X_train, X_test, y_train, y_test, 
    target_name="peak_altitude_ft",
    state_label="(Combined)"
)

# NJ
print("\n" + "="*70)
print("NJ STATE MODEL")
print("="*70)
X_nj = df_nj_reg[feature_cols].copy()
y_nj = df_nj_reg['peak_altitude_ft'].copy()
X_train_nj, X_test_nj, y_train_nj, y_test_nj = train_test_split(X_nj, y_nj, test_size=0.2, random_state=42)
results_pa_nj, _, _ = train_regression_models(
    X_train_nj, X_test_nj, y_train_nj, y_test_nj, 
    target_name="peak_altitude_ft",
    state_label="(NJ)"
)

# VT
print("\n" + "="*70)
print("VT STATE MODEL")
print("="*70)
X_vt = df_vt_reg[feature_cols].copy()
y_vt = df_vt_reg['peak_altitude_ft'].copy()
X_train_vt, X_test_vt, y_train_vt, y_test_vt = train_test_split(X_vt, y_vt, test_size=0.2, random_state=42)
results_pa_vt, _, _ = train_regression_models(
    X_train_vt, X_test_vt, y_train_vt, y_test_vt, 
    target_name="peak_altitude_ft",
    state_label="(VT)"
)


Spline Regression - peak_altitude_ft (Combined)
RMSE: 840.0280RMSE: 840.0280
R² Score: -0.4255

Random Forest - peak_altitude_ft (Combined)

R² Score: -0.4255

Random Forest - peak_altitude_ft (Combined)
RMSE: 655.5843
R² Score: 0.1318

XGBoost - peak_altitude_ft (Combined)
RMSE: 640.1966
R² Score: 0.1720

Top 10 Important Features (Random Forest):
                           Feature  Importance
4                      day_of_year    0.105952
13          surface_pressure (hPa)    0.088972
12                 cloud_cover (%)    0.082286
17  soil_temperature_0_to_7cm (°C)    0.063704
22         wind_direction_100m (°)    0.063666
15          wind_direction_10m (°)    0.062519
20           wind_gusts_10m (km/h)    0.054628
2                 lunar_irradiance    0.051748
10        relative_humidity_2m (%)    0.044573
21          wind_speed_100m (km/h)    0.043211

NJ STATE MODEL

Spline Regression - peak_altitude_ft (NJ)
RMSE: 655.5843
R² Score: 0.1318

XGBoost - peak_altitude_ft (Combined)
R

## 6. Classification Models: peak_direction

### Combined Data

In [38]:
# Encode direction labels
le = LabelEncoder()
y_dir_encoded = le.fit_transform(df_model['peak_direction'])

X = df_model[feature_cols].copy()
y = y_dir_encoded

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

results_pd_combined, scaler_pd, _ = train_classification_models(
    X_train, X_test, y_train, y_test, 
    target_name="peak_direction",
    state_label="(Combined)"
)

print(f"\nDirection classes mapping:")
for i, direction in enumerate(le.classes_):
    print(f"  {i}: {direction}")

# NJ
print("\n" + "="*70)
print("NJ STATE MODEL")
print("="*70)
y_nj_encoded = le.transform(df_nj['peak_direction'])
X_nj = df_nj[feature_cols].copy()
y_nj = y_nj_encoded
# Note: No stratification for state-specific splits due to rare direction classes
X_train_nj, X_test_nj, y_train_nj, y_test_nj = train_test_split(X_nj, y_nj, test_size=0.2, random_state=42)
results_pd_nj, _, _ = train_classification_models(
    X_train_nj, X_test_nj, y_train_nj, y_test_nj, 
    target_name="peak_direction",
    state_label="(NJ)"
)

# VT
print("\n" + "="*70)
print("VT STATE MODEL")
print("="*70)
y_vt_encoded = le.transform(df_vt['peak_direction'])
X_vt = df_vt[feature_cols].copy()
y_vt = y_vt_encoded
# Note: No stratification for state-specific splits due to rare direction classes
X_train_vt, X_test_vt, y_train_vt, y_test_vt = train_test_split(X_vt, y_vt, test_size=0.2, random_state=42)
results_pd_vt, _, _ = train_classification_models(
    X_train_vt, X_test_vt, y_train_vt, y_test_vt, 
    target_name="peak_direction",
    state_label="(VT)"
)


Random Forest Classifier - peak_direction (Combined)
Accuracy: 0.5599

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.50      0.08      0.14        12
           2       0.00      0.00      0.00         5
           3       1.00      1.00      1.00        66
           4       0.67      0.20      0.31        10
           5       0.47      0.67      0.55        42
           6       0.51      0.61      0.56        36
           7       0.00      0.00      0.00         3
           8       0.00      0.00      0.00         1
           9       0.35      0.32      0.33        25
          10       0.33      0.17      0.22        12
          11       0.12      0.07      0.09        14
          12       0.38      0.47      0.42        34
          13       0.48      0.74      0.58        34
          15       0.00      0.00      0.00         1
          16       0.50      0.17      0

## 7. Model Summary & Comparison

In [39]:
print("\n" + "="*70)
print("REGRESSION MODEL PERFORMANCE SUMMARY")
print("="*70)

summary_data = {
    'Target': [],
    'Region': [],
    'RF_RMSE': [],
    'RF_R2': [],
    'XGB_RMSE': [],
    'XGB_R2': [],
    'Best_Model': []
}

# peak_birds
for name, results in [('Combined', results_pb_combined), ('NJ', results_pb_nj), ('VT', results_pb_vt)]:
    rf_rmse = results['Random Forest']['rmse']
    rf_r2 = results['Random Forest']['r2']
    xgb_rmse = results['XGBoost']['rmse']
    xgb_r2 = results['XGBoost']['r2']
    
    summary_data['Target'].append('peak_birds')
    summary_data['Region'].append(name)
    summary_data['RF_RMSE'].append(f"{rf_rmse:.2f}")
    summary_data['RF_R2'].append(f"{rf_r2:.4f}")
    summary_data['XGB_RMSE'].append(f"{xgb_rmse:.2f}")
    summary_data['XGB_R2'].append(f"{xgb_r2:.4f}")
    summary_data['Best_Model'].append('RF' if rf_r2 > xgb_r2 else 'XGB')

# total_passed
for name, results in [('Combined', results_tp_combined), ('NJ', results_tp_nj), ('VT', results_tp_vt)]:
    rf_rmse = results['Random Forest']['rmse']
    rf_r2 = results['Random Forest']['r2']
    xgb_rmse = results['XGBoost']['rmse']
    xgb_r2 = results['XGBoost']['r2']
    
    summary_data['Target'].append('total_passed')
    summary_data['Region'].append(name)
    summary_data['RF_RMSE'].append(f"{rf_rmse:.2f}")
    summary_data['RF_R2'].append(f"{rf_r2:.4f}")
    summary_data['XGB_RMSE'].append(f"{xgb_rmse:.2f}")
    summary_data['XGB_R2'].append(f"{xgb_r2:.4f}")
    summary_data['Best_Model'].append('RF' if rf_r2 > xgb_r2 else 'XGB')

# peak_speed_mph
for name, results in [('Combined', results_ps_combined), ('NJ', results_ps_nj), ('VT', results_ps_vt)]:
    rf_rmse = results['Random Forest']['rmse']
    rf_r2 = results['Random Forest']['r2']
    xgb_rmse = results['XGBoost']['rmse']
    xgb_r2 = results['XGBoost']['r2']
    
    summary_data['Target'].append('peak_speed_mph')
    summary_data['Region'].append(name)
    summary_data['RF_RMSE'].append(f"{rf_rmse:.2f}")
    summary_data['RF_R2'].append(f"{rf_r2:.4f}")
    summary_data['XGB_RMSE'].append(f"{xgb_rmse:.2f}")
    summary_data['XGB_R2'].append(f"{xgb_r2:.4f}")
    summary_data['Best_Model'].append('RF' if rf_r2 > xgb_r2 else 'XGB')

# peak_altitude_ft
for name, results in [('Combined', results_pa_combined), ('NJ', results_pa_nj), ('VT', results_pa_vt)]:
    rf_rmse = results['Random Forest']['rmse']
    rf_r2 = results['Random Forest']['r2']
    xgb_rmse = results['XGBoost']['rmse']
    xgb_r2 = results['XGBoost']['r2']
    
    summary_data['Target'].append('peak_altitude_ft')
    summary_data['Region'].append(name)
    summary_data['RF_RMSE'].append(f"{rf_rmse:.2f}")
    summary_data['RF_R2'].append(f"{rf_r2:.4f}")
    summary_data['XGB_RMSE'].append(f"{xgb_rmse:.2f}")
    summary_data['XGB_R2'].append(f"{xgb_r2:.4f}")
    summary_data['Best_Model'].append('RF' if rf_r2 > xgb_r2 else 'XGB')

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

print("\n" + "="*70)
print("CLASSIFICATION MODEL PERFORMANCE SUMMARY (peak_direction)")
print("="*70)

clf_summary = {
    'Region': ['Combined', 'NJ', 'VT'],
    'RF_Accuracy': [
        f"{results_pd_combined['Random Forest']['accuracy']:.4f}",
        f"{results_pd_nj['Random Forest']['accuracy']:.4f}",
        f"{results_pd_vt['Random Forest']['accuracy']:.4f}"
    ],
    'XGB_Accuracy': [
        f"{results_pd_combined['XGBoost']['accuracy']:.4f}",
        f"{results_pd_nj['XGBoost']['accuracy']:.4f}",
        f"{results_pd_vt['XGBoost']['accuracy']:.4f}"
    ]
}

clf_df = pd.DataFrame(clf_summary)
print(clf_df.to_string(index=False))

print("\n✓ Model training complete")


REGRESSION MODEL PERFORMANCE SUMMARY
          Target   Region    RF_RMSE  RF_R2   XGB_RMSE XGB_R2 Best_Model
      peak_birds Combined  952633.20 0.3775  926752.78 0.4108        XGB
      peak_birds       NJ 1117717.98 0.2986 1149845.65 0.2577         RF
      peak_birds       VT  831522.21 0.2567  837085.03 0.2467         RF
    total_passed Combined 2123993.93 0.3572 2127783.63 0.3549         RF
    total_passed       NJ 2250235.30 0.2852 2433795.70 0.1638         RF
    total_passed       VT 1526624.72 0.1155 1573291.06 0.0606         RF
  peak_speed_mph Combined       6.99 0.3858       7.14 0.3587         RF
  peak_speed_mph       NJ       7.03 0.4234       7.16 0.4024         RF
  peak_speed_mph       VT       7.58 0.1379       7.17 0.2273        XGB
peak_altitude_ft Combined     655.58 0.1318     640.20 0.1720        XGB
peak_altitude_ft       NJ     525.41 0.3510     531.08 0.3369         RF
peak_altitude_ft       VT     712.83 0.1427     694.96 0.1851        XGB

CLASSIFICATI